<a href="https://colab.research.google.com/github/etgins/Mice_ASD_Detection/blob/main/audio_feature_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

----------------------------------------------
Written by Itamar Ginsberg & Alon Schreuer, October 2021



# **1. Import data and split labels**

---

In [2]:
from google.colab import drive
drive.mount('/content/drive')
import os
import numpy as np

# import dataset from xls
import pandas as pd

## COLAB WORK
## ITAMAR:
dataset = pd.read_excel('/content/drive/MyDrive/Project_A/Project_A_files/total_data UPDATED EM 020821.xlsx')
## ALON:
# dataset = pd.read_excel('/content/drive/MyDrive/total data.xlsx')

## ITAMAR:
dataset.to_csv (r'/content/drive/MyDrive/Project_A/Project_A_files/total_data UPDATED EM 020821.csv', index = None, header=True)
# dataset.dropna(axis = 0, how='any')

## ALON:
# dataset = pd.read_excel('/content/drive/MyDrive/total data.xlsx')

# extract only the relevant columns / features
X = dataset[["Name", "Start Point (Hz)", "End Point (Hz)", "Duration (s)", "Syllable number"]]
y = dataset[["Offspring Genotype"]]

# TODO - clean NaN values from dataset or X
X.dropna(axis = 0, how='any')
X


## TODO - encode text feature values
# from sklearn.preprocessing import LabelEncoder
# l1 = LabelEncoder()
# l1.fit(X['Name'])
# dataset.Name = l1.transform(dataset.Name)
# dataset

# X.describe()

Mounted at /content/drive


,Name,Start Point (Hz),End Point (Hz),Duration (s),Syllable number
0,17470O,NaN,NaN,NaN,NaN
1,17470O,57648.8278,45525.0820,00:00:00.059000,3.0
2,17470O,55558.5268,45525.0820,00:00:00.063000,2.0
3,17470O,54513.3763,43852.8412,00:00:00.063000,2.0
4,17470O,55976.5870,43434.7810,00:00:00.060000,2.0
...,...,...,...,...,...
4363,17472I,NaN,NaN,NaN,7.0
4364,17472I,NaN,NaN,NaN,7.0
4365,17472I,NaN,NaN,NaN,7.0
4366,17472I,NaN,NaN,NaN,1.0


# **2. Split data for different mice by name - unfinished**

---

In [8]:
# split dataset for each mouse

def take_data_by_name(name_searched, Data, name_list):
  # print(name_list)
  ind = name_list == name_searched
  # print(ind)
  matching_data = Data[ind]
  # print(matching_data)
  return matching_data


# find set of mouse names
name_column = dataset["Name"]
unique_name_set = set(name_column)
unique_name_list = list(unique_name_set)
# unique_name_list = unique_name_list[0:-1]   # first entry is column name
number_of_mice = len(unique_name_list)
print(unique_name_list)


['17474J', '08132N', nan, '08028L', '17489I', '08060M', '08103J', '08098I', '17477I', '17456K', '17490J', '17472I', '08105P', '08139L', '08134N(2)', '17455J', '17470O', '17491K', '08056J', '08102I', '08059O', '17454L', '08113G', '08111I', '08142K', '08114J', '08137B', '08121P', '08139L(2)', '08119O', '17482K', '08110N', '08107O', '08130I', '08027I', '08106M', '08136G', '08109B', '17479L', '08169M', '17453I', '08104K', '08112K', '08096P', '08101L', '17444K', '17479L ', '08057P', '17471P', '17484J', '17450L']


# **3. Extract features**

---

In [20]:
## --------------------------------------------------------
# TODO - for each mouse, take only data that belongs to it :
# note: not yet finished, for now use the data for all mice in X
## --------------------------------------------------------

## 1. example - for one mouse only:
## ------------------------------
# idx = 1
# mouse_split_data = take_data_by_name(unique_name_list[idx], X, name_column)
# print('mouse name:', unique_name_list[idx],', ' 'idx:', idx)
# mouse_split_data

## 2. trying for all mice:
## ---------------------
  # final data is made up of: 
    # avg. start freq. per syllable
    # avg. end freq. per syllable
    # avg. syllable duration per syllable
    # syllable distribution - percentage of each syllable
    # Bandwidth - TODO

num_of_syllables = 10
final_data_size = 4*num_of_syllables
# initialize finalized data variable: mouse_final_data
mouse_final_data = np.empty([number_of_mice,final_data_size])


for idx in range (1,2): #(number_of_mice):
  mouse_split_data = take_data_by_name(unique_name_list[idx], X, name_column)
  print('mouse name:', unique_name_list[idx],', ' 'idx:', idx)
  # print('matching data:', '\n',mouse_split_data)

  # calculate feature 1 - average start freq for each syllable (Alon)
  ## -------------------------------------------

  # TODO 
  # extract all syllables per mouse
  # create a dictionay with num of syllable as key and all it's start freqs arrange in list as a value
  # for every value in the dictionary compute the average
  # to it again in a loop for every mouse such that in the end I'll get an array with num of mouse as a key and the value is the dictionay with the average of start freqs per syllable (*think about better way to arrange the data)
  

  # extract all syllables per mouse
  start_freq = mouse_split_data["Start Point (Hz)"]
  # print(start_freq)
  

  # get the num of each syllable 


  # calculate feature 2 - average end freq for each syllable (Alon)
  ## -------------------------------------------


  # calculate feature 3 - syllable distribution (Itamar):
  ## -------------------------------------------
# get rid of NaN
  mouse_split_data = mouse_split_data.dropna(subset=["Syllable number"])

# take syll numbers
  syllable_num_row = np.transpose(np.array(mouse_split_data["Syllable number"]))
  # print(syllable_num_row)

# initialize distribution vector
  distribution = np.zeros([1,10])
  # print(distribution)

# increment distribution count
  for i in range(len(syllable_num_row)):
    # print("iteration", i)
    syll = int(syllable_num_row[i])
    # print("found syllable", syll)
    # print("old distribution:", distribution)
    distribution[0,syll-1] += 1 
    # print("new distribution:", distribution)

# normalize histogram to distribution
  distribution = distribution / distribution.sum()
  print("final distribution:", distribution)

# transfer results to mouse_final_data
  mouse_final_data[idx,21:31] = distribution


  # calculate feature 4 - average syllable duration for each syllable (Alon)
  ## -------------------------------------------

  # calculate feature 5 - Bandwidth (???) - TODO
  ## -------------------------------------------

  print(mouse_final_data[idx,:])


mouse name: 08132N , idx: 1
1.0
final distribution: [[0.13286713 0.13286713 0.15384615 0.01398601 0.00699301 0.00699301
  0.20979021 0.15384615 0.17482517 0.01398601]]
[0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
 0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
 9.76118064e-313 2.37151510e-322 0.00000000e+000 0.00000000e+000
 0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
 0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
 0.00000000e+000 1.32867133e-001 1.32867133e-001 1.53846154e-001
 1.39860140e-002 6.99300699e-003 6.99300699e-003 2.09790210e-001
 1.53846154e-001 1.74825175e-001 1.39860140e-002 0.00000000e+000
 0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
 0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000]
